In [ ]:
Write-Host 'Hello, PowerShell!' 
#-ForegroundColor Yellow -BackgroundColor Magenta

# Outline for this talk

1. Intro - 1 min
2. Overview (high level, fast) - 5 min
3. Basics - 24 min
4. Exercise - 30 min

## Overview

### Do Big Things
  - Make Sweeping File Changes
  - Wrangle Data
  - Use Web Apis (Make Web Apis, but that's another talk)
  - Automate repetetive tasks
  - Make a dashboard
  - Manage Environments
  
### Do it Wherever
  - From Windows 7 onward
  - From WSL
  - From a Remote Computer
  - From a Mac
  - From the Cloud

### Disclaimer

There are multiple options for syntax presented here; most often with a performance difference or specifics for the use case. 

_My rule of thumb_: At the shell, go with easiest to type. In a script, go with easiest to read- UNLESS the job at hand requires better performance or heightened security.

## The Basics

### Shell

A REPL - Read Eval Print Loop

- Tab completion. Begin typing and hit tab key for autocompletion
- Right Click is copy/paste
- Ctrl + r  search command history
- Profile
- Help and discoverability

In [ ]:
Get-ExecutionPolicy

Unrestricted


### Versions

5 is Windows-only. .NET Framework

7 is pwsh.exe. Cross-Plat, .NET Core, .NET 5+



### Execution policy

First experience with powershell: someone tells you to run a script. You run it and get an error. You now hate powershell.


In [ ]:
$PSVersionTable

### Scripts
- IDEs. ISE, VSCode, Vim
- Functions
- Modules

#### Debugger

- Local
- Remote


### Providers

Learn ONE subset of syntax and apply it in MANY cases

Navigate many systems just as you would the file system


In [ ]:
Get-PSProvider

Each of these providers can use commandlets like New-Item, Get-Item, Get-ChildItem. Think of it like this... you've used 'dir' in Windows cmd or... DOS? or maybe 'ls' in bash. 

What if you could look at any number of systems with the same concept?

Listing items with filtering, finding things you need. Grouping them and picking the parts you're interested in.


In [ ]:
#Get-ChildItem Variable:\

Get-ChildItem Alias:\

In [ ]:
$PSVersionTable | select *

## The Pipeline

Commandlet results are .Net objects.

Other shells and CLIs often return unstructured text that you'd have to parse.


## Discoverability

- Discover properties and use Where-Object to find the commandlet name of an alias.


In [ ]:
# Inspect like this:

$Items = Get-Command

$Items | Select-Object * -First 1


In [ ]:

# And like this:

$Items = Get-Command

$Items | Get-Member

# Can it be shortened to a one-liner....?



### WhatIf

Use '-Whatif' when you're uncertain.


In [ ]:
$myText = 'my fantastic text file'
New-Item -Path ~temp/myText.txt  -ItemType File -Force -WhatIf
Set-Content ~temp/myText.txt $myText -Verbose -WhatIf

## Queries

Filter (learn more later about filtering for performance) with Where or ?

In [ ]:
Get-History | Where-Object {$_.CommandLine -like '*Member*'} #| Format-List

## Loops

Loop over collections with ForEach or its alias %

There are also While(), Do...While, and Do...Until

In [ ]:
Get-Process | ForEach-Object {
   "Process Id is: $($_.Id)"
}

## Range Operator

The .. operator indicates a range. 1..5 means from 1 to 5. When you pipe a range to a foreach, each item in the pipeline is acted upon; 1, 2, 3, 4, 5.

In [ ]:
1..5 | % {Write-Host ($_ + $_)}

## Arrays

In [ ]:
$MyArray = @(
    'yellow',
    'white and teal',
    'lavendar or violet',
    'green',
    'brown'
)

$MyArray | Sort-Object

Access an element at a given index

In [ ]:
$MyArray[3]

Use properties and methods of the Array

e.g. IndexOf, Length

In [ ]:
$MyArray.IndexOf('brown')
#help about_Arrays

Add to an array

The `+=` is one simple, but potentially costly to "add" to an Array

It's creating a copy, so if dealing with large datasets... this is not ideal

In [ ]:
$MyArray | Measure-Object -Word

$MyArray += 'blue'

$MyArray | Measure-Object -Word

## Hash Tables

A hash table is like a Dictionary<string, object>.

It's a set of Key/Value pairs.

In [ ]:
$myHashTable = @{
    key = 'value'
    secondKey = 'secondvalue'
    numberKey = 5
    innerObjectKey = [pscustomobject]@{
        name = 'innerName'
        id = 7
    }
    arrayKey = @( 'first', 'second', 'third')
}
# A Modulo has been spotted 
$myHashTable.innerObjectKey.id % $myHashTable.numberKey

2


## PS custom objects

Keys are the Properties of the object

In [ ]:
$myCustomObject = [PSCustomObject] @{
    AmIAHashKey = 'Am I a value?'
    NoImAProperty = 'iAmAProperty'
    Weight = 44.22
    Height = 71
    DateOfBirth = (Get-Date).AddDays(-1)
}

$myCustomObject

### Data Wrangling

Build a custom data set from multiple data sources.

Important bits here: 

- Make a new List and add to it; Rather than `$Array += $AddedItem`
- Find a key by value in a hashtable - GETENUMERATOR!!!

In [ ]:
$vegetableColors = @{
    carrot = 'orange'
    onion = 'white'
    celery = 'light green'
    lettuce = 'green'
}

$animals = @(
    [pscustomobject]@{
        type = 'horse'
        name = 'wilbur'
        feedTime = 'am'
        vegColor = 'orange'
    },
    [pscustomobject]@{
        type = 'cow'
        name = 'mabel'
        feedTime = 'pm'
        vegColor = 'green'
    }
)

# C# - var feedingSchedule = new List<object>();
$feedingSchedule = New-Object System.Collections.Generic.List[object]

foreach($animal in $animals){
$feedingSchedule.Add(
    @{
        name = $animal.name
        time = if($animal.feedTime -eq 'am'){'morning'}elseif($animal.feedTime -eq 'pm'){'night'}
        food = $vegetableColors.GetEnumerator() | Where-Object { $_.Value -eq $animal.vegColor} | select -ExpandProperty Key
    }
    )
}

$feedingSchedule | Select-Object time, name, food

As C# has nuget packages and JS has NPM, PowerShell has Modules

Use what's out there, or write your own.

Testing - PowerShell tests are just more PowerShell. The defacto module to run tests is Pester and it's built in.

In [ ]:
Invoke-Pester